# The Sequential Model

Sequential model katmanların stack halinde sıralı olduğu bir yapıdadır. Sequential modeli bir input ve bir output olduğu durumlarda kullanabiliriz. 

In [1]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers

In [2]:
model = keras.Sequential([
    layers.Dense(2,activation="relu",name="layer1"),
    layers.Dense(3,activation="relu",name="layer2"),
    layers.Dense(4,name="layer3")
])

x = tf.ones((3,3))
y = model(x)

In [3]:
# Yukarıdaki modelin farklı şekilde yazımı ;
layer1 = layers.Dense(2,activation="relu",name="layer1")
layer2 = layers.Dense(3,activation="relu",name="layer2")
layer3 = layers.Dense(4,name="layers3")

x = tf.ones((3,3))
y = layer3(layer2(layer1(x)))

Sequential Model aşağıdaki durumlara uygun değildir.  
* Modelin birden fazla inputu ve outputu varsa, 
* Herhangi bir layerın birden fazla inputu ve outputu varsa
* Layer paylaşımı yapmak istersek 
* Liner olmayan bir topoloji kullanmak istersek 

# Sequential Model Oluşturma
Sequential model oluşturmak için constructor'a bir dizi layer göndeririz.

In [5]:
model = keras.Sequential([
    layers.Dense(2,activation="relu"),
    layers.Dense(3,activation="relu"),
    layers.Dense(4)
])
# Katmanlara .layer ile erişebiliriz. 
model.layers

In [7]:
# add() metodu kullanarakta Sequential model oluşturabiliriz
model = keras.Sequential()
model.add(layers.Dense(2,activation="relu"))
model.add(layers.Dense(3,activation="relu"))
model.add(layers.Dense(4))


In [8]:
# pop metoduyla son layerı çıkartabiliriz. Sequential model bir liste gibi davranır. 
model.pop()
print(len(model.layers))

2


In [9]:
# Ayrıca name parametresini de verebiliriz. 
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2,activation="relu",name="layer1"))
model.add(layers.Dense(3,activation="relu",name="layer2"))
model.add(layers.Dense(4,name="layer3"))

## İnput shape'in önceden verilmesi 
Genellikle, Kerasta tüm layerlar input shapelerini bilmelidir. Bu sayede weightleri oluşturabilirler. Shape olmayan bir layer oluşturursak ; 

In [11]:
layer = layers.Dense(3)
layer.weights

[]

In [13]:
# Weightler shape bilgisi alındıktan sonra oluşturulur. 
x = tf.ones((1,4))
y = layer(x)
layer.weights

[<tf.Variable 'dense_9/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.5799253 , -0.00989765,  0.37253153],
        [ 0.17615592, -0.62095666,  0.89335895],
        [-0.3366527 ,  0.07671583,  0.9188205 ],
        [ 0.20459676,  0.45761383, -0.3958488 ]], dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

Eğer input shape olmadan bir sequential model çalıştırırsak,model.weights ve .summary() metodlarında hata alırız. Weightler model ilk data gördüğünde oluşturulur. 

In [15]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # Weight yoktur

# bu durumda aşağıdaki satırlar hata verir
# model.weights
# model.summary()

x = tf.ones((1,4))
y = model(x)
print("Modeli çağırdıktan sonra weights sayısı : ",len(model.weights))

Modeli çağırdıktan sonra weights sayısı :  6


In [18]:
# Model oluştururken summary bilgisini görmek faydalı olacaktır. Bu sebeple Input objesi ile input shape belirtebiliriz. 
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2,activation="relu"))
print(model.summary())
# input objesi model.layers ile görünmez çünkü bir layer değildir.
model.layers

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# Alternatif olarak ilk katmana input_shape parametresi gönderebiliriz. 
model = keras.Sequential()
model.add(layers.Dense(2,activation="relu",input_shape=(4,)))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


## Debugging Workflow : add() + summary()
Sequential model oluştururken layerları add ile stackleyerek ilerleyip belirli aralıklarda summary ile kontrol etmemiz gerekebilir. İmage datası gönderdiğimizde datanın ne durumda olduğunu takip etmek önemli olabilir.

In [20]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))


model.summary() # output shape 40,40,32 --> hala downsampling yapmamıza olanak var. 


model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

model.summary()

# Şİmdi 4x4 feature map kaldı global max pooling uygulayabiliriz.
model.add(layers.GlobalMaxPooling2D())


model.add(layers.Dense(10))

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
__________________________

## Modeli oluşturduğumuzda ne yapacağız? 
Model mimarisi hazır olduğunda : 
* Model eğitimi, evaluate edilmesi ve tahminleme adımına geçeriz. 
* Modeli kaydedebiliriz. 


## Sequential modelde özellik çıkarımı 
Sequential model oluşturulduğunda Functional model gibi davranır. Bunun anlamı her bir katmanın input ve output özelliği vardır.

In [21]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [22]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Sequential Model ile Transfer Learning 
Transfer learning genellikle modelin alt katmanlarını dondurup üst katmanlarını eğiterek yapılır.  
Örneğin aşağıda bir sequential model oluşturalaım ve bunun sadece son katmanını eğitelim. Bunun için son katmana kadar olan layerların layer.trainable = False yapalım 

In [24]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# model.load_weights("") # burada eğitilmiş modelin ağırlıklarını yüklüyoruz. 

for layer in model.layers[:-1]:
    layer.trainable= False
    
# model.compile(...)
# model.fit()

In [ ]:
# Önceden eğitilmiş modelü oluşturalım
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Base modeli donduralım 
base_model.trainable = False

# Modelin en üstüne eğitilebilir bir layer ekleyelim 
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

#  compile 
# model.compile(...)
# model.fit(...)